### This notebook makes use of selenium and BeautifulSoup to scrape information relating to engineering job openings in Singapore on a government portal as at 31 Jul 2020. The scraped dataset can found within the `datasets` folder in this repository. Some data cleaning/ understanding/ exploration is done thereafter. 

In [1]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup
import csv
import shutil
import re
import time

In [2]:
import pandas as pd
import numpy as np

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [101]:
url = "https://vcf.mycareersfuture.sg/engine-room/section/engineering-jobs"
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)
driver.get(url)
engine_jobs = []

for count in range(0,1050): 
    time.sleep(2)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    for i in range(0,5):
        post = soup.find_all('p', class_='tile')[i].get_text().strip()
        title = post.split('\n')[0]
        company = post.split('\n')[1]
        posted_date = soup.find_all('p', class_='source')[i].get_text().strip()
        posted_date = posted_date.replace('Posted on\n\t\t\t\t\t\t\t', '')
        job_card = soup.find_all("div", class_="card2 job job-card")
        start = 'https:'
        end = ',,'
        job_post_url = 'https:'+ (str(job_card[i]).split(start))[1].split(end)[0]
        new = requests.get(job_post_url)
        job_url = new.text
        job_page = BeautifulSoup(job_url, 'html.parser')
        jd_chunk = str(job_page.find_all('p')).split('<p class="source">',1)[0]
        tag_list = ['<p><strong>','</strong>','</p>,','<p>','<br/>']
        for tag in tag_list:
            jd_chunk = jd_chunk.replace(tag, '')
        requirements_start = 'Salary</strong><br/>\n' 
        requirements_end = '</li>, <li><a data-featherlight'
        info_chunk = str(job_page.find_all('li'))
        try:
            chunking = (info_chunk.split(requirements_start))[1].split(requirements_end)[0]
            li_start = '</li>, <li>'
            requirements = (chunking.split(li_start,1))[1].replace(li_start,' ')
        except:
            requirements = 'N/A'
        
        info = str(job_page.find_all('ul',class_='info'))
        job_level_start = 'Job level</strong><br/>\n'
        job_level_end = '</li>'
        try:
            job_level = (info.split(job_level_start))[1].split(job_level_end)[0]
            job_level = job_level.strip()
        except:
            job_level = 'N/A'
            
        contract_type_start = 'Contract type</strong><br/>\n'
        try:
            contract_type = (info.split(contract_type_start))[1].split(job_level_end)[0]
            contract_type = contract_type.strip()
        except:
            contract_type = 'N/A'
            
        location_start = 'Location</strong><br/>\n'
        try:
            location = (info.split(location_start))[1].split(job_level_end)[0]
            location = location.strip()
        except:
            location = 'N/A'
            
        salary_start = 'Salary</strong><br/>\n'
        try:
            salary = (info.split(salary_start))[1].split(job_level_end)[0]
            salary = salary.strip()
        except:
            salary = 'N/A'
        
        try:
            closing_date = job_page.find('p',class_='source').get_text().replace('Closing on \n','').strip()
        except:
            closing_date = 'N/A'
        engine_jobs.append((title,company,posted_date,job_level,contract_type,location,salary,jd_chunk,
                           requirements,closing_date,job_post_url))
    
    print(count)
    elm = driver.find_element_by_class_name('next')
    elm.click()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: list index out of range

In [102]:
df = pd.DataFrame(np.array(engine_jobs))
df.columns = ['title','company','posted_date','job_level','contract_type','location','salary','job_desc','requirements',
             'closing_date','posting_url']

In [103]:
df.head()

,title,company,posted_date,job_level,contract_type,location,salary,job_desc,requirements,closing_date,posting_url
0,Manufacturing Engineer (Production Management)...,STMICROELECTRONICS PTE LTD,30 Jul 2020,Fresh/entry level,Internship/Traineeship,North-East,S$2500 - S$2500,[Our next technology can start with You! Join ...,Production daily Operations meetings to unders...,29 Aug 2020,https://vcf.mycareersfuture.sg/engine-room/job...
1,Senior Manufacturing Engineer (Manufacturing),MURATA ELECTRONICS SINGAPORE (PTE) LTD,30 Jul 2020,Senior Executive,"Permanent, Full Time",North,S$5000 - S$10000,[JOB DESCRIPTION: Department: Rationalization ...,Be responsible to achieve the machine requirem...,29 Aug 2020,https://vcf.mycareersfuture.sg/engine-room/job...
2,Engineering Technician 1 #SGUnitedTraineeships,ADVANCED MICRO DEVICES (SINGAPORE) PTE LTD,30 Jul 2020,Fresh/entry level,Internship/Traineeship,Central,S$1800 - S$1800,[THE ROLE: To assist or provide technical trou...,Provide basic and higher technical level of de...,29 Aug 2020,https://vcf.mycareersfuture.sg/engine-room/job...
3,Engineering Technician 1 (RMA Execution) #SGUn...,ADVANCED MICRO DEVICES (SINGAPORE) PTE LTD,30 Jul 2020,Fresh/entry level,Internship/Traineeship,Central,S$1800 - S$1800,[THE ROLE: To provide supportive functions to ...,"Perform Customer Returns (RMA) testing, record...",29 Aug 2020,https://vcf.mycareersfuture.sg/engine-room/job...
4,Supply Chain Engineering Rotation Program,MICRON SEMICONDUCTOR ASIA OPERATIONS PTE. LTD.,30 Jul 2020,Fresh/entry level,Full Time,North,S$3000 - S$6000,[Micron Technology’s vision is to transform ho...,Typically 3 years allowing 3 functional rotati...,29 Aug 2020,https://vcf.mycareersfuture.sg/engine-room/job...


In [104]:
len(df)

5010

In [105]:
df.to_csv('engine_jobs_310720.csv',index=False)

----------------------------------

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('engine_jobs_310720.csv')

In [3]:
df2 = df['salary'].str.split(' ', expand=True)
df2.columns = ['sal_min','dash','sal_max']
df2 = df2.drop('dash',axis=1)
df2['sal_min'] = df2['sal_min'].str.replace('S','')
df2['sal_min'] = df2['sal_min'].str.replace('$','')
df2['sal_max'] = df2['sal_max'].str.replace('S','')
df2['sal_max'] = df2['sal_max'].str.replace('$','')
df2['sal_min'] = pd.to_numeric(df2['sal_min'], errors = 'coerce')
df2['sal_max'] = pd.to_numeric(df2['sal_max'], errors = 'coerce')

In [5]:
df_final = df.join(df2,how='left')

In [6]:
df_final['posted_date'] = pd.to_datetime(df_final['posted_date'], errors='coerce')
df_final['closing_date'] = pd.to_datetime(df_final['closing_date'], errors='coerce')
df_final['days'] = (df_final['closing_date']- df_final['posted_date']).dt.days

In [7]:
df_final.head()

,title,company,posted_date,job_level,contract_type,location,salary,job_desc,requirements,closing_date,posting_url,sal_min,sal_max,days
0,Manufacturing Engineer (Production Management)...,STMICROELECTRONICS PTE LTD,2020-07-30,Fresh/entry level,Internship/Traineeship,North-East,S$2500 - S$2500,[Our next technology can start with You! Join ...,Production daily Operations meetings to unders...,2020-08-29,https://vcf.mycareersfuture.sg/engine-room/job...,2500.0,2500.0,30
1,Senior Manufacturing Engineer (Manufacturing),MURATA ELECTRONICS SINGAPORE (PTE) LTD,2020-07-30,Senior Executive,"Permanent, Full Time",North,S$5000 - S$10000,[JOB DESCRIPTION: Department: Rationalization ...,Be responsible to achieve the machine requirem...,2020-08-29,https://vcf.mycareersfuture.sg/engine-room/job...,5000.0,10000.0,30
2,Engineering Technician 1 #SGUnitedTraineeships,ADVANCED MICRO DEVICES (SINGAPORE) PTE LTD,2020-07-30,Fresh/entry level,Internship/Traineeship,Central,S$1800 - S$1800,[THE ROLE: To assist or provide technical trou...,Provide basic and higher technical level of de...,2020-08-29,https://vcf.mycareersfuture.sg/engine-room/job...,1800.0,1800.0,30
3,Engineering Technician 1 (RMA Execution) #SGUn...,ADVANCED MICRO DEVICES (SINGAPORE) PTE LTD,2020-07-30,Fresh/entry level,Internship/Traineeship,Central,S$1800 - S$1800,[THE ROLE: To provide supportive functions to ...,"Perform Customer Returns (RMA) testing, record...",2020-08-29,https://vcf.mycareersfuture.sg/engine-room/job...,1800.0,1800.0,30
4,Supply Chain Engineering Rotation Program,MICRON SEMICONDUCTOR ASIA OPERATIONS PTE. LTD.,2020-07-30,Fresh/entry level,Full Time,North,S$3000 - S$6000,[Micron Technology’s vision is to transform ho...,Typically 3 years allowing 3 functional rotati...,2020-08-29,https://vcf.mycareersfuture.sg/engine-room/job...,3000.0,6000.0,30


In [8]:
import pandas_profiling

In [9]:
df_final.profile_report()